In [1]:
# %matplotlib inline
import pandas as pd
from pandas import DataFrame
from pandas.io import sql
from pandas.tseries.offsets import *
import numpy as np
# import pylab as plt
import pymysql
# import seaborn
# from matplotlib import pyplot
from datetime import datetime,timedelta
import csv
import math
pd.options.display.float_format = '{:,.2f}'.format
import json

In [2]:
with open('config.json') as f:
    conf = json.load(f)

In [3]:
con = pymysql.connect(host=conf['host'], user=conf['user'], passwd=conf['password'], db=conf['database'])

In [4]:
ytd_trades = sql.read_sql("""
SELECT a.code, a.fundCode, a.orderType, a.side, a.swap, a.tradeDate, a.PB, g.commrate as gcomrate, d.brokerCommissionRate,
  # a.commission,
  (@commInUSD := a.commission*f.rate) AS commInUSD,
  (@commrate := IF (g.commrate IS NOT NULL, ROUND(g.commrate,4), ROUND(d.brokerCommissionRate,4))) AS CommRate,
  (@jpresearch := IF(b.currencyCode="JPY" AND c.instrumentType="EQ" AND  (@commrate=0.0015), 
      @commInUSD*11/15,0)*1.0) AS JPResearch,
      IF(b.currencyCode="JPY" AND c.instrumentType="EQ" AND (@commrate=0.0004), 
          @commInUSD, 
          0)*1.0 AS JPDis,
  (@clearing:=
  IF (c.instrumentType IN ("FU", "OP"), 
      IF(SUBSTRING(a.code, 1, 2) IN ("TP", "NK"), 500, 
        IF(SUBSTRING(a.code, 1, 2)="JP",50, 
		IF(SUBSTRING(a.code, 1, 2) IN ("HC", "HI"), 30, 0) 
	)
   ) * a.quantity *f.rate, 0  )) AS Clearing,
  IF(b.currencyCode="JPY" AND c.instrumentType="EQ" AND (@commrate=0.0004 OR @commrate=0),
      0,
      IF(b.currencyCode="JPY" AND c.instrumentType="EQ",
      @commInUSD - @jpresearch,
      IF(b.currencyCode="JPY" AND c.instrumentType IN ("FU", "OP"), 
      @commInUSD - @clearing,0) )) AS JPExec,
  d.brokerCode, c.instrumentType, b.currencyCode,
  IF(d.brokerCode="BXS", "Soft", IF(d.brokerCode="INSH", "Nomura",e.name)) AS brokerName,
  (@tax := CASE CONCAT(c.instrumentType, a.orderType, b.currencyCode)
            WHEN "EQBCNY" THEN 0.000098
            WHEN "EQSCNY" THEN 0.001098
            WHEN "EQBHKD" THEN 0.00108
            WHEN "EQSHKD" THEN 0.00108
            WHEN "EQSTWD" THEN 0.003
            WHEN "EQSKRW" THEN 0.003
            WHEN "EQBSGD" THEN 0.0004
            WHEN "EQSSGD" THEN 0.0004
            ELSE 0
            END
  ) AS tax,
  (@asiadeal := IF (b.currencyCode <> "JPY" AND c.instrumentType="EQ" AND d.brokerCommissionRate > 0.01,
                    a.gross * f.rate * (d.brokerCommissionRate - @tax ), 0)) AS asiaDeal,
  (@asiaResearch := IF(b.currencyCode <> "JPY" AND c.instrumentType="EQ" AND @asiadeal=0, 
    IF(d.brokerCommissionRate-@tax-0.0005>= 0, d.brokerCommissionRate-@tax-0.0005, 0) *f.rate*a.gross, 0)) AS asiaResearch,
  IF (b.currencyCode <> "JPY" AND c.instrumentType="EQ" AND @asiadeal=0,
    0.0005 * f.rate * a.gross,
    IF (b.currencyCode <> "JPY" AND c.instrumentType IN ("FU", "OP"), @commInUSD-@clearing, 0)
  ) AS asiaExecution,
  IF (a.swap="SWAP", @asiaResearch, 0)*1.0 AS HCSwaps
FROM t08Reconcile a
  INNER JOIN t01Instrument c ON a.code = c.quick
  INNER JOIN t02Currency b ON b.currencyID = c.currencyID
  INNER JOIN t08Reconcile d ON a.matchDoric = d.primaryID # a.primaryID = d.matchBrokers AND d.srcFlag ="D"
  INNER JOIN t06DailyCrossRate f ON f.priceDate=a.tradeDate AND f.base=b.currencyCode AND f.quote="USD"
  LEFT JOIN t02Broker e ON e.brokerCode = a.brokerCode
  left join (select a.code, a.orderType, a.side, a.swap, a.tradeDate, a.settleDate, a.brokerCode, MAX(a.brokerCommissionRate) as commrate
from t08Reconcile a
where a.status="A" and a.srcFlag="D" and a.tradeDate > '2015-12-31'
group by a.code, a.orderType, a.side, a.swap, a.tradeDate, a.settleDate, a.brokerCode
  ) g ON a.code=g.code and a.orderType=g.orderType and a.side=g.side and a.swap=g.swap and a.tradeDate=g.tradeDate
          and d.brokerCode=g.brokerCode
WHERE a.tradeDate>'2015-12-31' AND a.srcFlag="K"
ORDER BY a.tradeDate, a.code;
""", con, parse_dates=['tradeDate'],index_col='tradeDate' )

q2 = ytd_trades['2016-04-01':'2016-06-30']

In [5]:
# Asia commission budget is 2mm  USD per year
# Japan has gone through a few iterations but it's basically approximately 7.5 mm USD per year
jp_annual_commission_budget = 7500000  # usd
asia_annual_commission_budget = 2000000 # usd
jp_quarter_commission_budget = jp_annual_commission_budget / 4.0
asia_quarter_commission_budget = asia_annual_commission_budget / 4.0

jp_quarter_commission_budget = 1880911  # to make it same with old system

In [6]:
# broker rank for each quarter
data = {
    'brokers': ['BAML', 'Mizuho Securities', 'AdvancedResearch', 'Nomura', 'Japan Equity Research', 
                'Citi', 'Mitsubishi UFJ', 'Tokai', 'Ichiyoshi', 'SMBC Nikko', 
                'BNP', 'Deutsche', 'CLSA', 'Daiwa', 'Jefferies', 
                'CS', 'UBS', 'JP Morgan', 'Goldman Sachs', 'Okasan', 
                'MS', 'Macquarie'
               ],
    'rank': [6, 3, 21, 2, 17, 8, 7, 16, 15, 4, 20, 19, 22, 1, 14, 9, 12, 5, 9, 17, 9, 13],
    'research': map(lambda x: x*100, [0.0716, 0.0951, 0, 0.1014, 0,
                 0.0643, 0.0691, 0.0135, 0.0143, 0.0801,
                 0.008, 0.0096, 0.0006, 0.1104, 0.0193,
                 0.0565, 0.0463, 0.0724, 0.0564, 0.0116,
                 0.0563, 0.0298
                ])
}

jp_ranks_df = pd.DataFrame(data).set_index('brokers')
jp_ranks_df
# jp_ranks = pd.Series(jpranks)
# jp_ranks

,rank,research
brokers,,
BAML,6,7.16
Mizuho Securities,3,9.51
AdvancedResearch,21,0.00
Nomura,2,10.14
Japan Equity Research,17,0.00
Citi,8,6.43
Mitsubishi UFJ,7,6.91
Tokai,16,1.35
Ichiyoshi,15,1.43


In [7]:
(q2 #.assign(JPExec = lambda x: x.commInUSD-x.JPResearch if x.currencyCode=='JPY' else x.Clearing)
   .groupby(['brokerName', 'currencyCode'])
   .sum()
   .loc[(slice(None), 'JPY'),['commInUSD', 'JPResearch', 'JPExec', 'JPDis',  'Clearing']]
   .reset_index()
   .drop('currencyCode', axis=1)
   .set_index('brokerName')
   .merge(jp_ranks_df, how='right', left_index=True, right_index=True)
   .fillna(0)
   .sort_values(by='rank', axis=0)
  .assign( res_target = lambda df: df['research'] * jp_quarter_commission_budget / 100)
  .assign( balance_usd = lambda df: df['res_target']/3.0 - df['JPResearch'])
)

,commInUSD,JPResearch,JPExec,JPDis,Clearing,rank,research,res_target,balance_usd
brokers,,,,,,,,,
Daiwa,"187,746.84","108,228.73","39,355.90","40,162.21",0.00,1,11.04,"207,652.57","-39,011.20"
Nomura,"351,571.01","211,010.71","76,731.17","63,829.13",0.00,2,10.14,"190,724.38","-147,435.92"
Mizuho Securities,"301,400.69","173,813.54","63,204.92","64,382.23",0.00,3,9.51,"178,874.64","-114,188.66"
SMBC Nikko,"240,313.92","127,060.79","46,203.92","65,149.74",0.00,4,8.01,"150,660.97","-76,840.47"
JP Morgan,"188,536.20","110,683.77","40,248.65","37,603.78",0.00,5,7.24,"136,177.96","-65,291.12"
BAML,"194,580.94","120,373.65","44,395.29","29,188.93",623.07,6,7.16,"134,673.23","-75,482.57"
Mitsubishi UFJ,"179,081.10","100,139.81","36,414.48","42,526.82",0.00,7,6.91,"129,970.95","-56,816.16"
Citi,"262,376.55","149,397.79","54,326.47","58,652.30",0.00,8,6.43,"120,942.58","-109,083.59"
Goldman Sachs,"264,179.36","152,683.66","55,521.33","55,974.37",0.00,9,5.64,"106,083.38","-117,322.53"


In [8]:
(q2[q2['currencyCode'] != "JPY"]
 .groupby('brokerName')
 .sum()[['asiaResearch', 'asiaExecution', 'HCSwaps']]
# .reset_index()
 )

,asiaResearch,asiaExecution,HCSwaps
brokerName,,,
BAML,"37,099.69","14,675.42","25,947.95"
CLSA,"11,722.40","3,649.50",0.00
CS,"23,539.68","6,248.50","22,270.51"
Cantor,"11,062.13","3,694.47",0.00
Citi,"22,772.96","6,487.58","13,314.01"
Daiwa,"3,529.85",882.46,0.00
Deutsche,"3,778.27","1,261.82",0.00
Fubon Securities,"5,956.17","2,978.10",0.00
Goldman Sachs,"7,093.95","2,369.13",0.00


In [9]:
currency_mask = ytd_trades['currencyCode'] == 'JPY'
(ytd_trades.groupby(['brokerName', currency_mask])
           .sum()
           .assign(AsiaYTD = lambda x : x.asiaResearch + x.asiaExecution)
           .unstack()
           .fillna(0) [['JPResearch', 'JPExec', 'Clearing', 'commInUSD'
                        , 'AsiaYTD'
                       ]]
           .rename(columns={True: 'JP', False: 'Asia'})
         #.loc[(slice(None), 'JPY'), ['JPResearch', 'JPExec', 'commInUSD']]
)

JPResearch            JPExec             \
currencyCode                           Asia         JP   Asia         JP   
brokerName                                                                 
BAML                                   0.00 321,097.37   0.00 128,801.61   
BNP                                    0.00  97,447.18   0.00  35,435.34   
Barclays                               0.00   5,991.92   0.00   2,178.88   
CLSA                                   0.00   3,939.37   0.00   1,432.50   
CS                                     0.00 239,213.79   0.00  86,986.83   
Cantor                                 0.00       0.00   0.00       0.00   
Citi                                   0.00 308,272.21   0.00 112,098.98   
Daiwa                                  0.00 307,061.70   0.00 111,658.80   
Deutsche                               0.00  38,446.12   0.00  13,980.41   
Fubon Securities                       0.00       0.00   0.00       0.00   
Goldman Sachs                          0.00 332,786.74   0.00 121,013.36   
Ichiyoshi                              0.00  68,705.15   0.00  24,983.69   
JP Morgan                              0.00 237,927.89   0.00  86,519.23   
Jefferies                              0.00  53,110.07   0.00  19,312.75   
KGI Securities                         0.00       0.00   0.00       0.00   
MS                                     0.00 270,088.08   0.00 141,005.45   
Macquarie                              0.00  24,948.15   0.00   9,072.06   
Mitsubishi UFJ                         0.00 297,966.41   0.00 108,351.42   
Mizuho Securities                      0.00 426,204.88   0.00 154,983.59   
Nomura                                 0.00 424,381.82   0.00 154,320.66   
Okasan                                 0.00  46,937.70   0.00  17,068.25   
RHB OSK Securities Hong Kong Lim       0.00       0.00   0.00       0.00   
SMBC Nikko                             0.00 328,318.35   0.00 119,388.49   
Soft                                   0.00 284,879.09   0.00 103,592.40   
UBS                                    0.00 393,596.95   0.00 143,126.16   
Yuanta                                 0.00       0.00   0.00       0.00   

                                 Clearing            commInUSD             \
currencyCode                         Asia        JP       Asia         JP   
brokerName                                                                  
BAML                                 0.00 12,038.91 190,166.37 532,685.35   
BNP                                  0.00      0.00       0.00 162,716.46   
Barclays                             0.00      0.00       0.00   8,170.81   
CLSA                                 0.00      0.00  97,471.50   5,371.87   
CS                                   0.00      0.00 184,473.86 410,732.59   
Cantor                               0.00      0.00  47,042.90       0.00   
Citi                                 0.00      0.00 117,734.15 536,922.85   
Daiwa                                0.00      0.00  40,687.83 554,437.94   
Deutsche                             0.00      0.00  21,133.52  53,547.43   
Fubon Securities                     0.00      0.00  91,433.09       0.00   
Goldman Sachs                        0.00      0.00  47,843.16 563,653.66   
Ichiyoshi                            0.00      0.00       0.00  93,688.84   
JP Morgan                            0.00      0.00 116,372.48 374,295.14   
Jefferies                            0.00      0.00  44,046.10  72,422.82   
KGI Securities                       0.00      0.00  62,865.46       0.00   
MS                               4,934.43 51,282.62 144,414.66 471,094.65   
Macquarie                            0.00      0.00  44,740.64  34,020.21   
Mitsubishi UFJ                       0.00      0.00       0.00 539,498.79   
Mizuho Securities                    0.00      0.00       0.00 729,585.64   
Nomura                               0.00      0.00  80,137.36 747,504.60   
Okasan                               0.00      0.00      

In [10]:
# (ytd_trades.groupby(['brokerName', 'currencyCode'])
#            .sum()
#            #.assign(AsiaYTD = lambda x : x.asiaResearch + x.asiaExecution)
#            .unstack()
#            .fillna(0) [['commInUSD', 'JPResearch', 'JPExec', 'Clearing'
#             #            , 'AsiaYTD'
#                        ]]
#            .rename(columns={True: 'JP', False: 'Asia'})
#          #.loc[(slice(None), 'JPY'), ['JPResearch', 'JPExec', 'commInUSD']]
# )

In [11]:
(q2[q2['currencyCode'] != "JPY"]
   .groupby(['brokerName', 'currencyCode'])
   .sum()[['asiaResearch']]
   .unstack()
   .fillna(0)
)

asiaResearch                            
currencyCode              HKD      KRW       TWD      USD
brokerName                                               
BAML                 8,969.01     0.00 25,947.95 2,182.73
CLSA                 8,563.44     0.00  3,158.96     0.00
CS                   3,755.39     0.00 19,784.29     0.00
Cantor              11,062.13     0.00      0.00     0.00
Citi                 9,458.95     0.00 13,314.01     0.00
Daiwa                    0.00 3,529.85      0.00     0.00
Deutsche             3,778.27     0.00      0.00     0.00
Fubon Securities         0.00     0.00  5,956.17     0.00
Goldman Sachs        7,093.95     0.00      0.00     0.00
JP Morgan           22,018.55     0.00      0.00     0.00
Jefferies            9,775.09     0.00      0.00     0.00
KGI Securities           0.00     0.00  9,389.41     0.00
MS                   9,795.31     0.00 14,573.17 6,055.84
Macquarie            5,039.50     0.00      0.00     0.00
Nomura               8,399.89     0.00  3,538.63     0.00
UBS                 17,465.35     0.00      0.00     0.00
Yuanta                   0.00     0.00  9,676.40     0.00

In [12]:
# broker rank for each quarter
data = {
    'brokers': ['BAML', 'Mizuho Securities',  'Nomura', 'Japan Equity Research', 
                'Citi', 'Mitsubishi UFJ', 'Tokai', 'Ichiyoshi', 'SMBC Nikko', 
                'BNP', 'Deutsche', 'CLSA', 'Daiwa', 'Jefferies', 
                'CS', 'UBS', 'JP Morgan', 'Goldman Sachs', 'Okasan', 
                'MS', 'Macquarie'
               ],
    'rank': [6, 1, 2, 20, 9, 4, 18, 16, 3, 14, 19, 22, 5, 15, 11, 10, 12, 7, 17, 8, 21],
    'research': map(lambda x: x, [7.55, 10.26, 8.69, 0.81,
                 6.47, 7.95, 1.16, 1.36, 8.1,
                 2.19, 0.99, 0.13, 7.94, 1.46,
                 5.72, 8.18, 4.98, 6.99, 1.2,
                 6.86, 0.63
                ])
}

jp_ranks_df = pd.DataFrame(data).set_index('brokers')
jp_ranks_df

,rank,research
brokers,,
BAML,6,7.55
Mizuho Securities,1,10.26
Nomura,2,8.69
Japan Equity Research,20,0.81
Citi,9,6.47
Mitsubishi UFJ,4,7.95
Tokai,18,1.16
Ichiyoshi,16,1.36
SMBC Nikko,3,8.10


In [13]:
jp_quarter_commission_budget =10031527/4.0
jp_quarter_commission_budget

2507881.75

In [14]:
q1 = ytd_trades['2016-01-01':'2016-03-31']

For monthly balance need to divided res_target by 3 to find balance, but for quarter calculation, do not need to divide by 3.

In [15]:
# q1[q1['brokerCode']=='MSUS']
print("QTD Clearing : {}".format(q1['Clearing'].sum()))

QTD Clearing : 43394.1692735


In [16]:
q1[(q1['brokerName'] == 'Soft') & (q1['currencyCode'] == 'JPY')]['JPResearch'].sum()

174207.58411221573

In [17]:
jan = q1['2016-01']
print("Jan Soft= {}".format(jan[(jan['brokerName'] == 'Soft') ]['JPResearch'].sum()))

Jan Soft= 55914.9035038


In [43]:
jan_ms = jan[(jan['brokerName'] == 'Soft') ]['JPExec'].sum()
jan_ms
# '{:.2f}'.format(jan_ms)

'20332.69'

In [71]:
# q1 = ytd_trades['2016-02-05':'2016-02-08']
# q1 = ytd_trades['2016-02-22':'2016-02-22']
def format_2f(df):
    t = df.copy()
    columns = ['JPResearch', 'JPExec', 'JPDis', 'res_target', 'balance_usd', 'balance_jpy']
    t[columns] = t[columns].applymap(lambda x: '{:12,.0f}'.format(x))
    t['research'] = t['research'].apply(lambda x: '{:5.2f}%'.format(x))
    t['accrued'] = t['accrued'].apply(lambda x: '{:5.0f}%'.format(x))
    return t

usd_jpy = 112.27
# exec_target = pd.Series

table = (q1 #.assign(JPExec = lambda x: x.commInUSD-x.JPResearch if x.currencyCode=='JPY' else x.Clearing)
   .groupby(['brokerName', 'currencyCode'])
   .sum()
   .loc[(slice(None), 'JPY'),['JPResearch', 'JPExec', 'JPDis']]
   .reset_index()
   .drop('currencyCode', axis=1)
   .set_index('brokerName')
   .merge(jp_ranks_df, how='right', left_index=True, right_index=True)  # some names removed like Barclays and Softs
   .fillna(0)
   .sort_values(by='rank', axis=0)
  .assign( res_target = lambda df: df['research'] * jp_quarter_commission_budget / 100)
  .assign( balance_usd = lambda df: df['res_target'] - df['JPResearch'])
         .assign(balance_jpy = lambda df: df['balance_usd'] * usd_jpy)
  .assign( accrued = lambda df: (df['JPExec'] + df['JPDis'])*100/(df['JPExec'].sum() + df['JPDis'].sum()))
    .reset_index()
         .set_index('rank')
)[['research', 'res_target', 'JPResearch', 'JPExec', 'JPDis', 'balance_usd', 'balance_jpy', 'accrued']]
print('{:,.2f}'.format(table['JPExec'].sum()+table['JPDis'].sum()))
table.pipe(format_2f)

1,656,075.84


,research,res_target,JPResearch,JPExec,JPDis,balance_usd,balance_jpy,accrued
rank,,,,,,,,
1,10.26%,"257,309","252,391","91,779","84,015","4,917","552,068",11%
2,8.69%,"217,935","213,371","77,589","104,973","4,564","512,380",11%
3,8.10%,"203,138","201,258","73,185","84,691","1,881","211,165",10%
4,7.95%,"199,377","197,827","71,937","90,654","1,550","174,019",10%
5,7.94%,"199,126","198,833","72,303","95,555",293,"32,877",10%
6,7.55%,"189,345","200,724","84,406","41,559","-11,379","-1,277,481",8%
7,6.99%,"175,301","180,103","65,492","53,879","-4,802","-539,137",7%
8,6.86%,"172,041","218,012","100,281","8,718","-45,971","-5,161,200",7%
9,6.47%,"162,260","158,874","57,773","57,899","3,386","380,093",7%


In [ ]:
def calculate_commssion(quarter, ytd_trades, jp_ranks_df):
    
    return df

In [19]:
(q1[q1['currencyCode'] != "JPY"]
   .groupby(['brokerName'])
   .sum()[['asiaResearch', 'asiaExecution', 'HCSwaps']]
   .assign(asiaYTD = lambda x: x.asiaResearch + x.asiaExecution)
   # ['commission', 'commInUSD', 'JPResearch', 'JPExec', 'Clearing']
)

,asiaResearch,asiaExecution,HCSwaps,asiaYTD
brokerName,,,,
BAML,"50,964.80","16,184.29","24,123.68","67,149.09"
CLSA,"40,724.07","11,994.31",0.00,"52,718.38"
CS,"71,288.16","19,842.69","61,683.38","91,130.84"
Cantor,"11,825.46","3,949.27",0.00,"15,774.73"
Citi,"33,476.77","9,021.41","29,481.44","42,498.18"
Daiwa,"13,391.83","3,414.55",0.00,"16,806.38"
Deutsche,"6,503.71","2,172.05",0.00,"8,675.76"
Fubon Securities,"17,025.33","8,512.67",0.00,"25,538.00"
Goldman Sachs,"16,182.94","5,404.57",0.00,"21,587.51"


In [20]:
(q1[q1['currencyCode'] != "JPY"]
 .groupby(['brokerName', 'currencyCode'])
 .sum()[['asiaResearch']]
 .unstack()
 .fillna(0)
 )

asiaResearch                            \
currencyCode                              CNY       HKD       KRW   SGD   
brokerName                                                                
BAML                                 1,423.24 40,976.62      0.00  0.00   
CLSA                                     0.00 21,592.20      0.00  0.00   
CS                                       0.00 24,059.66 20,278.24  0.00   
Cantor                                   0.00 11,825.46      0.00  0.00   
Citi                                     0.00 11,936.42      0.00  0.00   
Daiwa                                    0.00    794.25 12,597.58  0.00   
Deutsche                                 0.00  6,503.71      0.00  0.00   
Fubon Securities                         0.00      0.00      0.00  0.00   
Goldman Sachs                            0.00 16,182.94      0.00  0.00   
JP Morgan                                0.00 30,018.46  3,424.08  0.00   
Jefferies                                0.00 11,655.02      0.00  0.00   
KGI Securities                           0.00      0.00      0.00  0.00   
MS                                       0.00  7,285.95 19,863.43  0.00   
Macquarie                                0.00 16,684.25      0.00 60.05   
Nomura                                   0.00  6,726.81  9,115.28  0.00   
RHB OSK Securities Hong Kong Lim         0.00    181.91      0.00  0.00   
UBS                                      0.00 28,731.08  5,530.22  0.00   
Yuanta                                   0.00      0.00      0.00  0.00   

                                            
currencyCode                           TWD  
brokerName                                  
BAML                              8,564.93  
CLSA                             19,131.87  
CS                               26,950.26  
Cantor                                0.00  
Citi                             21,540.35  
Daiwa                                 0.00  
Deutsche                              0.00  
Fubon Securities                 17,025.33  
Goldman Sachs                         0.00  
JP Morgan                             0.00  
Jefferies                             0.00  
KGI Securities                   14,552.11  
MS                               10,194.55  
Macquarie                             0.00  
Nomura                           10,544.87  
RHB OSK Securities Hong Kong Lim      0.00  
UBS                                   0.00  
Yuanta                           13,554.17

In [21]:
# q2jpm = ytd_trades #.loc['2017-04-03':]  #,ytd_trades['brokerCode'] == 'JPMF']
# q2jpm[(q2jpm['brokerCode'] == 'RHBO') & (q2jpm['currencyCode'] != 'JPY')]

In [22]:
# from IPython.display import display, HTML
#HTML(q1[(q1['currencyCode'] == 'HKD') & (q1['brokerCode'] == 'MERT')].to_html())
#HTML(q1[(q1['currencyCode'] != 'JPY') & (q1['brokerName'] == 'Nomura')].to_html())
# q1[(q1['currencyCode'] != 'JPY') & (q1['brokerName'] == 'UBS')].count()